In [34]:
# imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sklearn
import scipy.io
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [35]:
# Sub 1
sub1Xtrain = scipy.io.loadmat('sub1Xtrain.mat')
sub1Ytrain = scipy.io.loadmat('sub1Ytrain.mat')
sub1Xtest = scipy.io.loadmat('sub1Xtest.mat')
sub1Ytest = scipy.io.loadmat('sub1Ytest.mat')
xtrain1 = sub1Xtrain['trainData']
xtrain1 = pd.DataFrame(xtrain1)
ytrain1 = sub1Ytrain['results']
ytrain1 = pd.DataFrame(ytrain1)
xtest1 = sub1Xtest['trainData']
xtest1 = pd.DataFrame(xtest1)
ytest1 = sub1Ytest['results']
# Sub 2
sub2Xtrain = scipy.io.loadmat('sub2Xtrain.mat')
sub2Ytrain = scipy.io.loadmat('sub2Ytrain.mat')
sub2Xtest = scipy.io.loadmat('sub2Xtest.mat')
sub2Ytest = scipy.io.loadmat('sub2Ytest.mat')
xtrain2 = sub2Xtrain['trainData']
xtrain2 = pd.DataFrame(xtrain2)
ytrain2 = sub2Ytrain['results']
ytrain2 = pd.DataFrame(ytrain2)
xtest2 = sub2Xtest['trainData']
xtest2 = pd.DataFrame(xtest2)
ytest2 = sub2Ytest['results']
# Sub 4
sub4Xtrain = scipy.io.loadmat('sub4Xtrain.mat')
sub4Ytrain = scipy.io.loadmat('sub4Ytrain.mat')
sub4Xtest = scipy.io.loadmat('sub4Xtest.mat')
sub4Ytest = scipy.io.loadmat('sub4Ytest.mat')
xtrain4 = sub4Xtrain['trainData']
xtrain4 = pd.DataFrame(xtrain4)
ytrain4 = sub4Ytrain['results']
ytrain4 = pd.DataFrame(ytrain4)
xtest4 = sub4Xtest['trainData']
xtest4 = pd.DataFrame(xtest4)
ytest4 = sub4Ytest['results']
# Sub 5
sub5Xtrain = scipy.io.loadmat('sub5Xtrain.mat')
sub5Ytrain = scipy.io.loadmat('sub5Ytrain.mat')
sub5Xtest = scipy.io.loadmat('sub5Xtest.mat')
sub5Ytest = scipy.io.loadmat('sub5Ytest.mat')
xtrain5 = sub5Xtrain['trainData']
xtrain5 = pd.DataFrame(xtrain5)
ytrain5 = sub5Ytrain['results']
ytrain5 = pd.DataFrame(ytrain5)
xtest5 = sub5Xtest['trainData']
xtest5 = pd.DataFrame(xtest5)
ytest5 = sub5Ytest['results']

In [36]:
def TestSetError(Yset, Yp):
    correct = 0 
    n = len(Yset)
    for i in range(n):
        if(Yset[i] == Yp[i]):
            correct += 1 
    accuracy=correct/len(Yset)
    return accuracy

In [37]:
def evaluateModel(xtrain,ytrain,xtest,ytest):
    
    # LDA
    model = LinearDiscriminantAnalysis()
    cv = StratifiedKFold(n_splits=5, shuffle=False, random_state=None)
    # define grid
    grid = dict()
    grid['solver'] = ['svd', 'lsqr', 'eigen']
    # define search
    lda = GridSearchCV(model, grid, scoring='accuracy', cv=cv, n_jobs=-1)
    # perform the search
    results = lda.fit(xtrain, ytrain)
    # summarize
    print("LDA:")
    print('Mean Accuracy: %.3f' % results.best_score_)
    print('Config: %s' % results.best_params_)
    outputLDA = lda.predict(xtest)
    print("test accuracy =", TestSetError(ytest.ravel(), outputLDA))
    
    # SVM
    model = SVC()
    cv = StratifiedKFold(n_splits=5, shuffle=False, random_state=None)
    # define grid
    parameters = {'C': [1, 10], 'gamma': [0.001, 0.01, 1]}
    # define search
    svc = GridSearchCV(model, param_grid=parameters, scoring='accuracy', cv=cv, n_jobs=-1)
    # perform the search
    results = svc.fit(xtrain, ytrain)
    # summarize
    print("SVM:")
    print('Mean Accuracy: %.3f' % results.best_score_)
    print('Config: %s' % results.best_params_)
    outputSVM = svc.predict(xtest)
    print("test accuracy =", TestSetError(ytest.ravel(), outputSVM))
    
    # XGB
    param_test1 = {
    'max_depth':range(3,11,2),
    'min_child_weight':range(1,7,2)
    }
    xgb = GridSearchCV(
            estimator = XGBClassifier(
                            learning_rate = 0.1, 
                            n_estimators = 140, 
                            gamma = 0, 
                            subsample = 0.8, 
                            colsample_bytree = 0.8,
                            objective = 'binary:logistic',  
                            scale_pos_weight = 1, 
                            seed=42), 
            param_grid = param_test1, 
            scoring='accuracy',
            n_jobs=-1,
            cv=StratifiedKFold(n_splits=5, shuffle=False, random_state=None)
            )

    xgb.fit(xtrain,ytrain)
    print("XGB:")
    print(xgb.best_params_,)
    print('Mean Accuracy: %.3f' %  xgb.best_score_)
    outputXGB = xgb.predict(xtest)
    print("test accuracy =", TestSetError(ytest.ravel(), outputXGB))
    
    # CAT boost
    cat_param_test1 = {
    'depth':range(1,10,2),
    'min_data_in_leaf': range(1,10,2)
    }
    cat_gsearch1 = GridSearchCV(
                estimator = CatBoostClassifier(
                                learning_rate = 0.1, 
                                iterations = 200,
                                loss_function = 'Logloss', # try crossEntropy later
                                border_count = 32,
                                eval_metric="AUC",
                                random_seed=42), 
                param_grid = cat_param_test1, 
                scoring='accuracy',
                n_jobs=-1,
                # use shuffle to verity validity of hyperparameters 
                cv= StratifiedKFold(n_splits=5, shuffle=False, random_state=None)
                )

    cat_gsearch1.fit(xtrain,ytrain)
    print("CATboost:")
    print(cat_gsearch1.best_params_)
    print('Mean Accuracy: %.3f' % cat_gsearch1.best_score_)
    outputCAT = cat_gsearch1.predict(xtest)
    print("test accuracy =", TestSetError(ytest.ravel(), outputCAT))

In [38]:
evaluateModel(xtrain1,ytrain1,xtest1,ytest1)

/Users/allan/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/allan/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LDA:
Mean Accuracy: 0.794
Config: {'solver': 'svd'}
test accuracy = 0.5919701213818861
SVM:
Mean Accuracy: 0.812
Config: {'C': 1, 'gamma': 0.001}
test accuracy = 0.5854341736694678


/Users/allan/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


XGB:
{'max_depth': 5, 'min_child_weight': 1}
Mean Accuracy: 0.796
test accuracy = 0.5779645191409897
0:	total: 4.92ms	remaining: 980ms
1:	total: 8.13ms	remaining: 805ms
2:	total: 11.7ms	remaining: 768ms
3:	total: 15.1ms	remaining: 741ms
4:	total: 18.1ms	remaining: 706ms
5:	total: 20.7ms	remaining: 670ms
6:	total: 23.6ms	remaining: 650ms
7:	total: 26.7ms	remaining: 641ms
8:	total: 30.3ms	remaining: 644ms
9:	total: 33.6ms	remaining: 639ms
10:	total: 36.4ms	remaining: 625ms
11:	total: 39.1ms	remaining: 613ms
12:	total: 43.9ms	remaining: 632ms
13:	total: 48.2ms	remaining: 641ms
14:	total: 51.3ms	remaining: 632ms
15:	total: 55.1ms	remaining: 633ms
16:	total: 58.6ms	remaining: 631ms
17:	total: 65ms	remaining: 657ms
18:	total: 67.9ms	remaining: 647ms
19:	total: 70.9ms	remaining: 638ms
20:	total: 73.8ms	remaining: 629ms
21:	total: 80.9ms	remaining: 655ms
22:	total: 85.6ms	remaining: 659ms
23:	total: 88.9ms	remaining: 652ms
24:	total: 96.1ms	remaining: 673ms
25:	total: 100ms	remaining: 672ms
26

In [39]:
evaluateModel(xtrain2,ytrain2,xtest2,ytest2)

/Users/allan/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LDA:
Mean Accuracy: 0.790
Config: {'solver': 'svd'}
test accuracy = 0.6630434782608695


/Users/allan/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


SVM:
Mean Accuracy: 0.800
Config: {'C': 1, 'gamma': 0.001}
test accuracy = 0.644927536231884


/Users/allan/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


XGB:
{'max_depth': 7, 'min_child_weight': 3}
Mean Accuracy: 0.794
test accuracy = 0.6539855072463768
0:	total: 480us	remaining: 95.7ms
1:	total: 1.65ms	remaining: 164ms
2:	total: 2.13ms	remaining: 140ms
3:	total: 2.63ms	remaining: 129ms
4:	total: 3.12ms	remaining: 122ms
5:	total: 3.62ms	remaining: 117ms
6:	total: 4.11ms	remaining: 113ms
7:	total: 4.63ms	remaining: 111ms
8:	total: 5.12ms	remaining: 109ms
9:	total: 5.6ms	remaining: 106ms
10:	total: 6.1ms	remaining: 105ms
11:	total: 6.59ms	remaining: 103ms
12:	total: 7.12ms	remaining: 102ms
13:	total: 7.66ms	remaining: 102ms
14:	total: 8.14ms	remaining: 100ms
15:	total: 8.63ms	remaining: 99.3ms
16:	total: 9.12ms	remaining: 98.2ms
17:	total: 9.66ms	remaining: 97.7ms
18:	total: 10.2ms	remaining: 96.8ms
19:	total: 10.7ms	remaining: 96ms
20:	total: 11.2ms	remaining: 95.3ms
21:	total: 11.7ms	remaining: 94.4ms
22:	total: 12.2ms	remaining: 93.8ms
23:	total: 12.7ms	remaining: 93.3ms
24:	total: 13.2ms	remaining: 92.5ms
25:	total: 13.7ms	remaining:

In [40]:
evaluateModel(xtrain4,ytrain4,xtest4,ytest4)

/Users/allan/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LDA:
Mean Accuracy: 0.854
Config: {'solver': 'svd'}
test accuracy = 0.7598684210526315


/Users/allan/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


SVM:
Mean Accuracy: 0.778
Config: {'C': 1, 'gamma': 0.001}
test accuracy = 0.6398026315789473


/Users/allan/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


XGB:
{'max_depth': 9, 'min_child_weight': 3}
Mean Accuracy: 0.844
test accuracy = 0.7516447368421053
0:	total: 2.02ms	remaining: 403ms
1:	total: 3.94ms	remaining: 391ms
2:	total: 5.82ms	remaining: 382ms
3:	total: 7.99ms	remaining: 391ms
4:	total: 11.6ms	remaining: 451ms
5:	total: 15.1ms	remaining: 489ms
6:	total: 17.4ms	remaining: 480ms
7:	total: 20.9ms	remaining: 501ms
8:	total: 23.2ms	remaining: 492ms
9:	total: 25.7ms	remaining: 488ms
10:	total: 27.9ms	remaining: 479ms
11:	total: 29.9ms	remaining: 468ms
12:	total: 32ms	remaining: 460ms
13:	total: 33.8ms	remaining: 449ms
14:	total: 35.3ms	remaining: 435ms
15:	total: 37ms	remaining: 425ms
16:	total: 38.5ms	remaining: 414ms
17:	total: 40ms	remaining: 405ms
18:	total: 41.7ms	remaining: 397ms
19:	total: 43.2ms	remaining: 389ms
20:	total: 44.7ms	remaining: 381ms
21:	total: 46.2ms	remaining: 374ms
22:	total: 47.7ms	remaining: 367ms
23:	total: 49.7ms	remaining: 364ms
24:	total: 51.7ms	remaining: 362ms
25:	total: 53.3ms	remaining: 357ms
26:	t

In [41]:
evaluateModel(xtrain5,ytrain5,xtest5,ytest5)

LDA:
Mean Accuracy: 0.770
Config: {'solver': 'svd'}
test accuracy = 0.6139927623642943


/Users/allan/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/allan/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


SVM:
Mean Accuracy: 0.792
Config: {'C': 1, 'gamma': 0.001}
test accuracy = 0.5922798552472859


/Users/allan/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


XGB:
{'max_depth': 7, 'min_child_weight': 1}
Mean Accuracy: 0.774
test accuracy = 0.6079613992762364
0:	total: 640us	remaining: 127ms
1:	total: 1.55ms	remaining: 154ms
2:	total: 2.18ms	remaining: 143ms
3:	total: 2.74ms	remaining: 134ms
4:	total: 3.32ms	remaining: 129ms
5:	total: 3.88ms	remaining: 125ms
6:	total: 4.43ms	remaining: 122ms
7:	total: 5.02ms	remaining: 120ms
8:	total: 5.56ms	remaining: 118ms
9:	total: 6.1ms	remaining: 116ms
10:	total: 6.65ms	remaining: 114ms
11:	total: 7.27ms	remaining: 114ms
12:	total: 7.83ms	remaining: 113ms
13:	total: 8.37ms	remaining: 111ms
14:	total: 8.96ms	remaining: 110ms
15:	total: 9.5ms	remaining: 109ms
16:	total: 10ms	remaining: 108ms
17:	total: 10.5ms	remaining: 106ms
18:	total: 11ms	remaining: 105ms
19:	total: 11.5ms	remaining: 104ms
20:	total: 12.2ms	remaining: 104ms
21:	total: 12.7ms	remaining: 103ms
22:	total: 13.3ms	remaining: 102ms
23:	total: 13.8ms	remaining: 101ms
24:	total: 14.3ms	remaining: 100ms
25:	total: 14.8ms	remaining: 99.3ms
26:	t